# petite demonstration du filtre gaussien (séparable)

In [ ]:
# settings for display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# turn off matplotlib figure axes, else we need to call axes('off') everytime
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.set_cmap('gray') 
mpl.rc('axes.spines',top=False,bottom=False,left=False,right=False)
mpl.rc(('xtick','ytick'),color=(1,1,1,0))
mpl.rc('font', size=20)

# import necessarry libs
from imageio import imread
import numpy as np
from scipy.signal import convolve2d as conv2

# define gaussian filter
def f_gauss(size, sigma):
    # gaussian filter
    x, y = np.mgrid[-size[0]//2 + 1:size[0]//2 + 1, -size[1]//2 + 1:size[1]//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()

In [ ]:
im = imread('img.jpg').astype(float)/255;
from skimage.color import rgb2gray
im = rgb2gray(im);
print(im.shape)

# pour que la différence de temps soit observable, 
# il faut des images assez grosses
from skimage.transform import resize
# im = resize(im, [5000 NaN]);
# filtre 2-D
f2d = f_gauss([25,25], 4);
# filtre 1-D
f1dv = f_gauss([1, 25], 4);

# filtrage avec le filtre 2D
import time
tic=time.time(); 
im2d = conv2(im, f2d, 'same'); 
print(time.time()-tic)

# filtrage avec le filtre 1D, et sa transposée
tic=time.time(); 
im1d = conv2(conv2(im, f1dv, 'same'), f1dv.T, 'same'); 
print(time.time()-tic);

In [ ]:
# plot
%matplotlib notebook
fig = plt.figure(figsize=(10,5), num='Gaussian')
ax1 = fig.add_subplot(1, 3, 1)
ax1.imshow(im2d); ax1.set_title('Filtre 2-D')
ax2 = fig.add_subplot(1, 3, 2, sharex = ax1, sharey = ax1)
ax2.imshow(im1d); ax2.set_title('2 filtres 1-D');
ax3 = fig.add_subplot(1, 3, 3, sharex = ax1, sharey = ax1)
fmap = ax3.imshow(np.abs(im1d-im2d)*100, vmin=0, vmax=1); ax3.set_title('100x difference');
fig.colorbar(fmap, ax=ax3,fraction=0.046, pad=0.04)
plt.tight_layout()